In [1]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 586.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [15]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from groq import Groq
import json
import os
from dotenv import load_dotenv
import requests
import geocoder
from PIL import Image
from io import BytesIO

In [16]:
load_dotenv()
LLAMA_API_KEY = os.getenv('LLAMA_KEY')
LLAMA_DEPLOYMENT = os.getenv('LLAMA_DEPLOYMENT_VERSION')
AZURE_MAPS_KEY = '6tTgAoW6inJAdK4BWCgzjbyG7ULZjU45dKiRFitLfHjUPcu1o9ZnJQQJ99AIACYeBjFuPQS6AAAgAZMPBkmY'

llama_client = Groq(api_key=LLAMA_API_KEY)

In [47]:
!python --version

Python 3.10.12


In [108]:
def load_documents_from_paths(file_paths):
    all_documents = []
    for file_path in file_paths:
        print(f"Loading document: {file_path}")
        loader = UnstructuredFileLoader(file_path)
        try:
            documents = loader.load()
            all_documents.extend(documents)
        except Exception as e:
            print(f"Error loading document {file_path}: {e}")
    return all_documents

In [109]:
docs=load_documents_from_paths(['usa_food.pdf','usa_food2.pdf'])

Loading document: usa_food.pdf
Loading document: usa_food2.pdf


In [110]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [111]:
texts = text_splitter.split_documents(docs)

In [71]:
embeddings = HuggingFaceEmbeddings()

In [112]:
persist_directory = "doc_db"

In [113]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [114]:
# retriever
retriever = vectordb.as_retriever()

In [115]:
# llm from groq
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0
)

In [116]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [103]:
def search_poi(latitude,longitude, poi_type):
    """Get an interest point from a location and a type of place"""
    url = 'https://atlas.microsoft.com/search/poi/json'
    params = {
        'api-version': '1.0',
        'subscription-key': AZURE_MAPS_KEY,
        'query': poi_type,
        'limit': 1,
        'lat': latitude,
        'lon': longitude,
        'radius':2000
    }

    response = requests.get(url, params=params)
    poi_name = response.json()['results'][0]['poi']['name']
    poi_address = response.json()['results'][0]['address']['freeformAddress']
    if response.status_code == 200:
        return json.dumps({"name": poi_name,"address":poi_address})
    else:
        print(f"Full response: {response.text}")
        raise Exception(f"Error searching for POI: {response.status_code}")

def get_static_map_with_poi(latitude,longitude):
    url = "https://atlas.microsoft.com/map/static/png"
    params = {
        'api-version': '1.0',
        'subscription-key': AZURE_MAPS_KEY,
        'layer': 'basic',  # Puedes elegir diferentes capas de mapa (basic, hybrid, etc.)
        'style': 'main',  # Estilo del mapa
        'zoom': 15,  # Nivel de zoom
        'center': f"{longitude},{latitude}",
        'height': 512,  # Altura de la imagen
        'width': 512,   # Anchura de la imagen
        'pins': f"default||{longitude} {latitude}"  # Marcador en el POI
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        #image.show()  # Muestra la imagen

        image.save("map_with_poi.png")  # Guarda la imagen localmente
        print('The POI image with the location was saved successfully')
    else:
        print(f"Error obteniendo el mapa: {response.status_code}")

def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})


def run_conversation(user_prompt):
    messages=[
        {
            "role": "system",
            "content": "You are an assistant and tourist guide who provides information and geospatial information. Use the search_poi function to get an interest point and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "search_poi",
                "description": "Get an interest point from a location and a type of place",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "latitude": {
                            "type": "string",
                            "description": "latitude from the geolocalization data such as '-17.7865' ",
                        },
                        "longitude": {
                            "type": "string",
                            "description": "longitude from the geolocalization data such as '-63.1812' ",
                        },
                        "poi_type": {
                            "type": "string",
                            "description": "The poi(point of interest) type such as 'restaurant'",
                        }
                    },
                    "required": ["latitude","longitude","poi_type"],
                },
            },
        },
    ]
    response = llama_client.chat.completions.create(
        model=LLAMA_DEPLOYMENT,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "search_poi": search_poi,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                latitude=function_args.get("latitude"),
                longitude=function_args.get("longitude"),
                poi_type=function_args.get("poi_type")

            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = llama_client.chat.completions.create(
            model=LLAMA_DEPLOYMENT,
            messages=messages
        )
        return second_response.choices[0].message.content



def get_current_gps_coordinates():
    g = geocoder.ip("me")#this function is used to find the current information using our IP Add

    if g.latlng is not None: #g.latlng tells if the coordiates are found or not
        return g.latlng
    else:
        return None



In [129]:

    coordinates = get_current_gps_coordinates()
    if coordinates is not None:
        latitude, longitude = coordinates
        print(f"Your current GPS coordinates are:")
        print(f"Latitude: {latitude}")
        print(f"Longitude: {longitude}")
    else:
        print("Unable to retrieve your GPS coordinates.")

    poi_type = 'restaurant'
    lat=40.7577
    lon=-74.2029
    try:
        user_prompt = f"Find the nearest {poi_type} to this location latitude: {latitude}, longitude:{longitude} and provide name of the place, something about the place such as history or popularity, address and its geolocalization (the format for show geolocation must be like this geolocalization: GEO[latitude,longitude])."
        out=run_conversation(user_prompt)
        print(out)
        lat_poi=out[out.find('GEO')+4:out.find('GEO')+11]
        lon_poi=out[out.find('GEO')+13:out.find('GEO')+19]
        print(f'POS:[{lat_poi}, {lon_poi}]')
        get_static_map_with_poi(lat_poi,lon_poi)
        print("Do you want know something about Usa food, in order to buy food correctly")
        print('#######options#######')
        print('1. yes')
        print('2. no')
        option=int(input('Select an option: '))
        if option==1:
          print()
          print('#########options#######')
          print('1. ask something')
          print('2. exit')
          while True:
            option=int(input('option: '))
            if option==1:
              query = input("What do you want to know?: ")
              response = qa_chain.invoke({"query": query})
              print(response["result"])
            else:
              print('bye!')
              break
        else:
          print('bye!')

    except Exception as e:
        print(f"An error occurred: {str(e)}")


Your current GPS coordinates are:
Latitude: 38.8951
Longitude: -77.0364
I've used the provided restaurant name to gather more information. According to my sources, Two Continents Restaurant has a rich history in Washington D.C. It was established in 1985 and has been a popular spot for locals and tourists alike. The restaurant is known for its eclectic menu featuring dishes from around the world.

Now, let me provide you with the restaurant's geolocation in the requested format:

GEO[38.8957, -77.0339]

Please note that the geolocation I provided is the exact location of the restaurant, which is slightly different from the initial location you provided. This is because the initial location you provided might have been a general area or a city center, rather than the exact location of the restaurant.

As your guide, I hope you enjoy your dining experience at Two Continents Restaurant!
POS:[38.8957, -77.03]
The POI image with the location was saved successfully
Do you want know something